#### WPE & WAPE Calculation

In [1]:
!pip install uritools

In [2]:
import pandas as pd
import numpy as np
import boto3
import pyarrow.parquet as pq
import s3fs
import math
import yaml

import sys
sys.path.append('../../')
from src.utils import read_multipart_parquet_s3, write_df_to_parquet_on_s3,read_csv_s3

In [3]:
with open('monitor_whitelist.yaml') as config_file:
    monitor_wl = yaml.full_load(config_file)

##### Fetch data

In [4]:
df_tree = read_multipart_parquet_s3('fcst-workspace/forecast-cn/fcst-refined-demand-forecast-dev',
                                    'global/model_week_tree', prefix_filename='part-')
df_tree = df_tree[(df_tree.week_id == 202214)]

In [5]:
df_mrp = read_multipart_parquet_s3('fcst-workspace/forecast-cn/fcst-refined-demand-forecast-dev',
                                    'global/model_week_mrp', prefix_filename='part-')
df_mrp = df_mrp[(df_mrp.week_id == 202214)][['model_id','is_mrp_active']]

In [16]:
fcst_bi_static_feat = pd.read_csv('s3://fcst-data-exchange-dev/demand-forecast/cn/dashboard/tableau/postgres/fcst_bi_static_feat_w/fcst_bi_static_feat_w_202214.csv')

In [17]:
fcst_bi_static_feat.shape

(10912, 17)

In [18]:
fcst_bi_static_feat = fcst_bi_static_feat.merge(df_tree[['model_id','family_id']],on='model_id')
fcst_bi_static_feat = fcst_bi_static_feat[(fcst_bi_static_feat.family_id.isin(monitor_wl['pre_selection_family']))]

In [19]:
fcst_bi_static_feat = fcst_bi_static_feat.merge(df_mrp,on='model_id',how='left')

In [20]:
fcst_bi_static_feat[['model_id', 'wpe_reach_target', 'wape_reach_target',
       'univers_label', 'department_label', 'family_label', 'rs_email',
       'AW2021_CodeA', 'tier', 'model_label', 'ec_tag', 'selling_length',
       'aw_seasonality', 'annual_seasonality', 'wpe_cutoff_ok_pct',
       'wape_cutoff_ok_pct','is_mrp_active', 'update_time']].to_csv('fcst_bi_static_feat_w.csv',index=False)

In [44]:
#fcst_bi_dynamic_feat = read_csv_s3('fcst-workspace', 
#                                   'forecast-cn/fcst-data-exchange-dev/dashboard/tableau/total_cutoff/fcst_bi_dynamic_feat.csv')

In [64]:
fcst_bi_dynamic_feat = pd.DataFrame()
for cutoff in list(range(202101,202153)) + list(range(202201,202214)):
    print(cutoff)
    df = pd.read_csv(f"s3://fcst-data-exchange-dev/demand-forecast/cn/dashboard/tableau/postgres/fcst_bi_dynamic_feat/fcst_bi_dynamic_feat_{cutoff}.csv")
    fcst_bi_dynamic_feat = fcst_bi_dynamic_feat.append(df)

202101
202102
202103
202104
202105
202106
202107
202108
202109
202110
202111
202112
202113
202114
202115
202116
202117
202118
202119
202120
202121
202122
202123
202124
202125
202126
202127
202128
202129
202130
202131
202132
202133
202134
202135
202136
202137
202138
202139
202140
202141
202142
202143
202144
202145
202146
202147
202148
202149
202150
202151
202152
202201
202202
202203
202204
202205
202206
202207
202208
202209
202210
202211
202212
202213


In [65]:
fcst_bi_dynamic_feat = fcst_bi_dynamic_feat.merge(df_tree[['model_id','family_id']],on='model_id')
fcst_bi_dynamic_feat = fcst_bi_dynamic_feat[(fcst_bi_dynamic_feat.family_id.isin(monitor_wl['pre_selection_family']))]

In [66]:
fcst_bi_dynamic_feat[['model_id','week_id','weekly_average_price','num_store_following']]\
.to_csv('fcst_bi_dynamic_feat.csv',index=False)

In [47]:
#fcst_bi_forecast_result = read_csv_s3('fcst-workspace', 
#                                      'forecast-cn/fcst-data-exchange-dev/dashboard/tableau/total_cutoff/fcst_bi_forecast_result.csv')

In [59]:
fcst_bi_forecast_result = pd.DataFrame()
for cutoff in list(range(202101,202153)) + list(range(202201,202215)):
    print(cutoff)
    df = pd.read_csv(f"s3://fcst-data-exchange-dev/demand-forecast/cn/dashboard/tableau/postgres/fcst_bi_forecast_result/fcst_bi_forecast_result_{cutoff}.csv")
    fcst_bi_forecast_result = fcst_bi_forecast_result.append(df)

202101
202102
202103
202104
202105
202106
202107
202108
202109
202110
202111
202112
202113
202114
202115
202116
202117
202118
202119


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


202120
202121
202122
202123
202124
202125
202126
202127
202128
202129
202130
202131
202132
202133
202134
202135
202136
202137
202138
202139
202140
202141
202142
202143
202144
202145
202146
202147
202148
202149
202150
202151
202152
202201
202202
202203
202204
202205
202206
202207
202208
202209
202210
202211
202212
202213
202214


In [60]:
fcst_bi_forecast_result[(fcst_bi_forecast_result.algorithm == 'realized_sales') \
                        & (fcst_bi_forecast_result.qty_tag == 'sales_w')].quantity.sum()

#51537265/152819899

162520547

In [47]:
fcst_bi_forecast_result = fcst_bi_forecast_result.merge(df_tree[['model_id','family_id']],on='model_id')
fcst_bi_forecast_result = fcst_bi_forecast_result[(fcst_bi_forecast_result.family_id.isin(monitor_wl['pre_selection_family']))]

AttributeError: 'DataFrame' object has no attribute 'family_id'

In [ ]:
fcst_bi_forecast_result[['model_id','week_id','algorithm','qty_tag','quantity']]\
.to_csv('fcst_bi_forecast_result.csv',index=False)

In [40]:
fcst_bi_forecast_result.groupby(by='algorithm',as_index=False).agg({"week_id":"max"})

,algorithm,week_id
0,AI,202223
1,APO_GD,202218
2,APO_STAT,202218
3,realized_sales,202213


In [52]:
#fcst_bi_forecast_performance = read_csv_s3('fcst-workspace', 
#                                           'forecast-cn/fcst-data-exchange-dev/dashboard/tableau/total_cutoff/fcst_bi_forecast_performance_a.csv')

In [61]:
fcst_bi_forecast_performance = pd.DataFrame()
for cutoff in list(range(202101,202153)) + list(range(202201,202211)):
    print(cutoff)
    df = pd.read_csv(f"s3://fcst-data-exchange-dev/demand-forecast/cn/dashboard/tableau/postgres/fcst_bi_forecast_performance_a/fcst_bi_forecast_performance_a_{cutoff}.csv")
    fcst_bi_forecast_performance = fcst_bi_forecast_performance.append(df)

202101
202102
202103
202104
202105
202106
202107
202108
202109
202110
202111
202112
202113
202114
202115
202116
202117
202118
202119
202120
202121
202122
202123
202124
202125
202126
202127
202128
202129
202130
202131
202132
202133
202134
202135
202136
202137
202138
202139
202140
202141
202142
202143
202144
202145
202146
202147
202148
202149
202150
202151
202152
202201
202202
202203
202204
202205
202206
202207
202208
202209
202210


In [62]:
fcst_bi_forecast_performance = fcst_bi_forecast_performance.merge(df_tree[['model_id','family_id']],on='model_id')
fcst_bi_forecast_performance = fcst_bi_forecast_performance[(fcst_bi_forecast_performance.family_id.isin(monitor_wl['pre_selection_family']))]

In [63]:
fcst_bi_forecast_performance.to_csv('fcst_bi_forecast_performance_a.csv',index=False)

In [14]:
fcst_bi_static_feat[['model_id']].drop_duplicates().to_csv('model_id.csv',index=False)

#### Check

In [6]:
df_sales = read_multipart_parquet_s3('fcst-workspace/forecast-cn/fcst-refined-demand-forecast-dev',
                                    'global/model_week_sales', prefix_filename='part-')

In [9]:
df_sales[(df_sales.model_id == 8554046)].groupby(by='week_id').agg({"sales_quantity":"sum"})

,sales_quantity
week_id,
201952,144
202001,434
202002,477
202003,554
202004,552
...,...
202208,503
202209,396
202210,409
